# EchoPro Python Workflow <a class="tocSkip">

# Import libraries and configure the Jupyter notebook

In [1]:
# libraries used in the Notebook
import matplotlib.pyplot as plt
import numpy as np
import folium

# Python version of EchoPro
import EchoPro

# Allows us to grab the SemiVariogram class so we can use its models
from EchoPro.semivariogram import SemiVariogram as SV

# Allows us to easily use matplotlib widgets in our Notebook
%matplotlib widget

# Set up  EchoPro for a specific survey year

## Initialize EchoPro object  using configuration files

* `initialization_config.yml` -- parameters independent of survey year
* `survey_year_2019_config.yml` -- parameters specific to survey year
* `source` -- Define the region of data to use e.g. US, CAN, US & CAN
* `exclude_age1` -- States whether age 1 hake should be included in analysis.

In [2]:
%%time
survey_2019 = EchoPro.Survey(init_file_path='../config_files/initialization_config.yml',
                             survey_year_file_path='../config_files/survey_year_2019_config.yml',
                             source=3, 
                             exclude_age1=True)

A check of the initialization file needs to be done!
A check of the survey year file needs to be done!
CPU times: user 16.6 ms, sys: 3.29 ms, total: 19.9 ms
Wall time: 21.9 ms


## Load and process input data 
* This data is stored in `survey_2019`

In [3]:
%%time 
survey_2019.load_survey_data() #file_type='biological')

CPU times: user 1.85 s, sys: 32.8 ms, total: 1.88 s
Wall time: 1.93 s


In [4]:
survey_2019.specimen_df.head()

,Sex,Length,Weight,Age
Haul,,,,
1,1,24.0,0.08,1.0
1,1,23.0,0.06,1.0
1,1,22.0,0.06,1.0
1,1,21.0,0.06,1.0
1,2,22.0,0.06,1.0


## Compute the normalized biomass density
* The biomass density is stored in `survey_2019`

In [5]:
%%time
survey_2019.compute_biomass_density()

CPU times: user 180 ms, sys: 4.16 ms, total: 184 ms
Wall time: 181 ms


In [6]:
survey_2019.final_biomass_table.head()

,Latitude,Longitude,Stratum,Spacing,normalized_biomass_density
Transect,,,,,
1,34.397267,-121.143005,1,10.0,0.0
1,34.397391,-121.133196,1,10.0,0.0
1,34.397435,-121.123057,1,10.0,0.0
1,34.397394,-121.112871,1,10.0,0.0
1,34.397437,-121.102888,1,10.0,0.0


# Jolly-Hampton CV Analysis

* Compute the mean of the Jolly-Hampton CV value on data that has not been Kriged
* Note: the algorithm used to compute this value is random in nature

In [7]:
%%time
CV_JH_mean = survey_2019.run_cv_analysis(kriged_data=False)

CPU times: user 1.62 s, sys: 22.2 ms, total: 1.64 s
Wall time: 1.67 s


In [8]:
print(f"Mean Jolly-Hampton CV: {CV_JH_mean:.4f}")

Mean Jolly-Hampton CV: 0.1338


# Obtain Kriging Mesh Data

## Access Kriging mesh object
* Reads mesh data files specified by `survey_2019` 

In [9]:
krig_mesh = survey_2019.get_kriging_mesh()

### Plot the Mesh, Transects and smoothed isobath contour

* Generate interactive map using the Folium package
* Mesh points are in gray
* Transect points are represented by a changing color gradient
* Smoothed contour points (200m isobath) are in blue 

In [ ]:
# Plot the full mesh
fmap = krig_mesh.plot_points(krig_mesh.mesh_gdf, name='mesh',
                             color='gray')

# Plot the transect points
fmap = krig_mesh.plot_points(epro_2019.final_biomass_table, name='transect',
                             fmap=fmap, cmap_column='Transect', color='hex')

# Plot smoothed contour points 
fmap = krig_mesh.plot_points(krig_mesh.smoothed_contour_gdf, name='contour',
                             fmap=fmap, color='blue')

# display the map
folium.LayerControl().add_to(fmap)
fmap

## Apply coordinate transformations
* Longitude transformation
* Lat/Lon to distance

### Transect points

In [ ]:
krig_mesh.apply_coordinate_transformation(
    survey_2019.final_biomass_table, 
    transform_type='transect'
)

### Mesh points

In [ ]:
krig_mesh.apply_coordinate_transformation(
    krig_mesh.mesh_gdf, 
    transform_type='mesh'
)

In [ ]:
# plot the transformed mesh points 
plt.plot(krig_mesh.mesh_transf_df.x_mesh, 
         krig_mesh.mesh_transf_df.y_mesh, 'r*', markersize=1.25)
plt.show()

# Compute biomass density Semi-Variogram and fit a model

* Compute the normalized semi-variogram using the normalized biomass density
* Fit a model to the semi-variogram values

## Compute the semi-variogram

### Initialize semi-variogram calculation
* Transformed transect points
* Parameters specific to semi-variogram algorithm

In [ ]:
semi_vario = survey_2019.get_semi_variogram(
    krig_mesh,
    params=dict(nlag=30, lag_res=0.002)
)

### Compute the normalized semi-variogram

In [ ]:
%%time
semi_vario.calculate_semi_variogram()

## Fit a model to the semi-variogram

* A widget to easily fit the model

In [ ]:
semi_vario.view_semi_variogram()

# Perform Ordinary Kriging of biomass density

* transformed mesh points
* semi-variogram model
* normalized biomass density 

## Initialize Kriging routine

In [ ]:
kriging_params = dict(
    # kriging parameters
    k_max=10,
    k_min=3,
    R=0.0226287,
    ratio=0.001,
    
    # parameters for semi-variogram model
    s_v_params={'nugget': 0.0, 'sill': 0.95279, 'ls': 0.0075429,
                'exp_pow': 1.5, 'ls_hole_eff': 0.0},
    
    # grab appropriate semi-variogram model
    s_v_model=SemiVariogram.generalized_exp_bessel
)

# initalize kriging routine
krig = survey_2019.get_kriging(kriging_params)

## Perform Kriging
* Also generates total biomass at mesh points

In [ ]:
%%time
mesh_krigbiomass_gdf = krig.run_biomass_kriging(krig_mesh)

In [ ]:
print(f"Total Kriged Biomass Estimate: {mesh_krigbiomass_gdf.krig_biomass_vals.sum():.2f} kmt")

## Plot Kriged Biomass estimate in kmt

In [ ]:
# only plot non-zero mesh points 
# mesh_krigbiomass_gt0_gdf = mesh_krigbiomass_gdf[mesh_krigbiomass_gdf["krig_biomass_vals"] > 0]
# krig.plot_kriging_results(mesh_krigbiomass_gt0_gdf, krig_fieldname="krig_biomass_vals")

# plot all mesh points
krig.plot_kriging_results(mesh_krigbiomass_gdf, krig_fieldname="krig_biomass_vals")